This notebook creates the following tables:

- roster (final_team_roster.csv)
- teams

## Imports

In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from pyalex import Works, Authors, Sources, Institutions, Topics, Publishers, Funders
import os
from pathlib import Path
from tqdm import tqdm 
import json

In [ ]:
from pyalex import config

config.email = "rde6mn@virginia.edu"
config.max_retries = 5
config.retry_backoff_factor = 0.1
config.retry_http_codes = [429, 500, 503]

## Combine all separate team roster excels into one df

In [ ]:
# Set the path to the folder containing the Excel files
folder_path = Path("Data/Just Teams")

# Get a list of all Excel files in the folder
excel_files = list(folder_path.glob("*.xlsx"))

combined_df = pd.DataFrame()

# Iterate through each file
for idx, file in enumerate(excel_files):
    # Read only the first 7 columns
    df = pd.read_excel(file, usecols=range(7), sheet_name=0)
    
    # For the first file, keep the original headers
    if idx == 0:
        combined_df = df.copy()
        combined_df['Team'] = file.stem  # Add 'Team' column using filename
    else:
        df.columns = combined_df.columns[:-1]  # Reuse headers from first file (excluding 'Team')
        df['Team'] = file.stem
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# Final combined dataframe: combined_df


## Obtain OpenAlexID value using ORCID value

In [ ]:

# Assuming Authors() returns a dictionary-like object you can index with ORCID URLs
# If it's a function that does live API calls, make sure it's efficient or cached
author_data = Authors()  # preload or initialize API access

# Add new column to hold OpenAlex ID suffix
combined_df['OpenAlex_ID'] = None

# Dictionary to store full JSON objects for matched authors
orcid_json_dict = {}

# Iterate over DataFrame rows
for idx, row in tqdm(combined_df.iterrows(), total=combined_df.shape[0]):
    orcid_id = row['ORCID']
    full_name = f"{row['First Name']} {row['Last Name']}".strip()

    orcid_url = f"https://orcid.org/{orcid_id}"
    
    try:
        # Fetch data
        author_json = author_data[orcid_url]

        # Compare names (case-insensitive match)
        if author_json.get('display_name', '').strip().lower() == full_name.lower():
            # Extract suffix of OpenAlex ID
            openalex_suffix = author_json['id'].split('/')[-1]

            # Store in DataFrame
            combined_df.at[idx, 'OpenAlex_ID'] = openalex_suffix

            # Store full JSON object in dictionary
            orcid_json_dict[orcid_id] = author_json

    except Exception as e:
        print(f"Error processing ORCID {orcid_id}: {e}")
        continue


## Add missing lab values

In [ ]:
cleaned_df = pd.read_csv("Data/cleaned.csv")

In [ ]:
cleaned_df['Lab'] = str(cleaned_df['Lab'])
cleaned_df['First_Name'] = str(cleaned_df['First_Name'])
cleaned_df['Last_Name'] = str(cleaned_df['Last_Name'])
combined_df['Lab'] = str(combined_df['Lab'])
combined_df['First Name'] = str(combined_df['First Name'])
combined_df['Last Name'] = str(combined_df['Last Name'])

In [ ]:
cleaned_df.columns = cleaned_df.columns.str.strip()
combined_df.columns = combined_df.columns.str.strip()

# Create lowercase copies for matching (don't modify originals)
combined_df['_fn_lc'] = combined_df['First Name'].astype(str).str.strip().str.lower()
combined_df['_ln_lc'] = combined_df['Last Name'].astype(str).str.strip().str.lower()
combined_df['_orcid_lc'] = combined_df['ORCID'].astype(str).str.strip().str.lower()
combined_df['_lab_lc'] = combined_df['Lab'].astype(str).str.strip().str.lower()

cleaned_df['_fn_lc'] = cleaned_df['First_Name'].astype(str).str.strip().str.lower()
cleaned_df['_ln_lc'] = cleaned_df['Last_Name'].astype(str).str.strip().str.lower()
cleaned_df['_orcid_lc'] = cleaned_df['ORCID'].astype(str).str.strip().str.lower()
cleaned_df['_lab_lc'] = cleaned_df['Lab'].astype(str).str.strip().str.lower()

# Count missing Lab values before
missing_before = combined_df['Lab'].isna().sum()
print(f"Missing 'Lab' values before update: {missing_before}")

# Fill missing Lab
missing_lab_mask = combined_df['Lab'].isna()

for idx, row in combined_df[missing_lab_mask].iterrows():
    fn, ln, orcid = row['_fn_lc'], row['_ln_lc'], row['_orcid_lc']

    match = cleaned_df[
        (cleaned_df['_fn_lc'] == fn) &
        (cleaned_df['_ln_lc'] == ln) &
        (cleaned_df['_orcid_lc'] == orcid)
    ]

    if not match.empty:
        combined_df.at[idx, 'Lab'] = match.iloc[0]['Lab']





## Pull orcid from cleaned_df if missing from combined_df

In [ ]:
# Ensure 'OtherORCID' column exists
if 'OtherORCID' not in combined_df.columns:
    combined_df['OtherORCID'] = pd.NA

# Process only rows with missing OpenAlex_ID
missing_openalex_mask = combined_df['OpenAlex_ID'].isna()

for idx, row in combined_df[missing_openalex_mask].iterrows():
    fn = row['_fn_lc']
    ln = row['_ln_lc']
    lab = row['_lab_lc']

    # Find match in cleaned_df
    match = cleaned_df[
        (cleaned_df['_fn_lc'] == fn) &
        (cleaned_df['_ln_lc'] == ln) &
        (cleaned_df['_lab_lc'] == lab)
    ]

    if not match.empty:
        combined_df.at[idx, 'OtherORCID'] = match.iloc[0]['ORCID']


In [ ]:
combined_df.drop(columns=['_fn_lc', '_ln_lc', '_lab_lc', '_orcid_lc'], inplace=True)

## For rows missing OpenAlexID, cross-ref with cleaned_df. If first name, last name and lab match, use the ORCID value to pull an OpenAlexID

In [ ]:
# Ensure OtherAlex_ID column exists
combined_df['OtherAlex_ID'] = pd.NA

# Instantiate the API object
authors_api = Authors()

# Filter rows where OpenAlex_ID is missing and OtherORCID is not equal to ORCID
mask = combined_df['OpenAlex_ID'].isna() & (
    combined_df['OtherORCID'].notna() & (combined_df['OtherORCID'] != combined_df['ORCID'])
)

for idx, row in combined_df[mask].iterrows():
    orcid = row['OtherORCID']
    try:
        author_json = authors_api[f"https://orcid.org/{orcid}"]
        openalex_id_full = author_json.get('id', '')

        if openalex_id_full.startswith("https://openalex.org/"):
            openalex_id_suffix = openalex_id_full.split("https://openalex.org/")[-1]
            combined_df.at[idx, 'OtherAlex_ID'] = openalex_id_suffix
    except Exception as e:
        print(f"Error processing ORCID {orcid} at index {idx}: {e}")


## Combine OpenAlex_ID and OtherAlex_ID to create a column with verified ids

In [ ]:
combined_df['VerifiedAlex_IDS'] = combined_df['OpenAlex_ID'].fillna(combined_df['OtherAlex_ID'])

## For rows missing VerifiedAlex_IDS, use the ORCID to pull from OpenAlex API without checking for name matches

In [ ]:
# Ensure column exists
combined_df['UnverifiedAlex_ID'] = pd.NA

# Instantiate OpenAlex API
authors_api = Authors()

# Identify target rows
mask = combined_df['VerifiedAlex_IDS'].isna() & combined_df['ORCID'].notna()

# Loop through applicable rows
for idx, row in combined_df[mask].iterrows():
    orcid = row['ORCID']
    try:
        author_json = authors_api[f"https://orcid.org/{orcid}"]
        openalex_url = author_json.get('id', '')
        if openalex_url.startswith("https://openalex.org/"):
            suffix = openalex_url.split("https://openalex.org/")[-1]
            combined_df.at[idx, 'UnverifiedAlex_ID'] = suffix
    except Exception as e:
        print(f"Failed for ORCID {orcid} at index {idx}: {e}")


## Combine verified and unverified open alex ids 

In [ ]:
combined_df['AllOpenAlex_ID'] = combined_df['VerifiedAlex_IDS'].fillna(combined_df['UnverifiedAlex_ID'])

## Clean Team Name

In [ ]:
## combined_df = pd.read_csv("combined_teams_with_all_openalex.csv")

In [3]:
combined_df['Team'] = combined_df['Team'].str.replace('Team ', '')

## Merge supplement results onto roster

In [5]:
supplement_df = pd.read_excel("Supplement Grants by Team.xlsx")

In [7]:
combined_df = combined_df.merge(supplement_df, on='Team', how='left')

## Merge rounds onto roster

In [10]:
rounds_df = pd.read_csv("ASAP Rounds - Sheet1.csv")

In [11]:
combined_df = combined_df.merge(rounds_df, on='Team', how='left')

In [13]:
combined_df.to_csv("final_team_roster.csv", index=False)